In [1]:
%pwd

'/media/kirti/Dev/GenAI/E2EMedicalChatBotWithRAG/notebooks'

In [2]:
%cd ..

/media/kirti/Dev/GenAI/E2EMedicalChatBotWithRAG


In [6]:
from E2EMedicalChatBotWithRAG.models.llm_model import LLMAssistant
llm_class = LLMAssistant()
llm = llm_class.get_model()

[2025-09-20 16:06:22,145|(INFO)| File: helper | Message: Loaded environment variable: GROQ_API_KEY]


In [4]:
from langchain_groq import ChatGroq
# import asyncio
# from langchain.callbacks.streaming_aiter import AsyncIteratorCallbackHandler

/media/kirti/Dev/GenAI/E2EMedicalChatBotWithRAG/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# # handler = AsyncIteratorCallbackHandler()

# llm = ChatGroq(
#             model="llama-3.1-8b-instant",
#             temperature=0.7,
#             max_tokens=512,
#             streaming=True 
#         #     callbacks=[handler]              # 🔑 enable streaming
#     )

In [7]:
from langchain_core.prompts import ChatPromptTemplate

In [2]:
%cd ..

/media/kirti/Dev/GenAI/E2EMedicalChatBotWithRAG


In [4]:
from src.E2EMedicalChatBotWithRAG.vector_database.redis_db import RedisDB
retriever = RedisDB().get_retriever()

[2025-09-20 16:42:49,234|(INFO)| File: SentenceTransformer | Message: Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2]
[2025-09-20 16:45:37,167|(INFO)| File: embedding_model | Message: Successfully loaded embedding model: sentence-transformers/all-MiniLM-L6-v2]
[2025-09-20 16:45:37,239|(INFO)| File: index | Message: Index already exists, not overwriting.]
[2025-09-20 16:45:37,282|(INFO)| File: index | Message: Index already exists, not overwriting.]
[2025-09-20 16:45:37,324|(WARNING)| File: redis_db | Message: Index medical-chatbot is empty or does not exist.]
[2025-09-20 16:45:37,365|(INFO)| File: index | Message: Index already exists, not overwriting.]


In [20]:
retriever

VectorStoreRetriever(tags=['RedisVectorStore', 'HuggingFaceEmbeddings'], vectorstore=<langchain_redis.vectorstores.RedisVectorStore object at 0x72d53780f830>, search_kwargs={'k': 3})

In [21]:
retriever.invoke("hey")

[]

In [10]:
system_prompt = """You are a helpful AI medical assistant that helps people find information.
You are given the following extracted parts of a long document and a question. Provide a conversational answer
based on the context provided.
If you don't know the answer, just say that you don't know. Don't try to make up an answer.
Always include relevant sources in your answer.
Make sure the answer is in detail and more than 100 words.

Context: {context}

If the question is out of context, politely inform them that you are tuned to only answer questions that are related to the context.
"""

In [11]:
prompt = ChatPromptTemplate.from_messages(
            [
                ("system", system_prompt),
                ("user", "{input}")
            ]        )

In [12]:
qna_chain =   prompt | llm

In [13]:
from langchain.chains import create_retrieval_chain
rag_chain = create_retrieval_chain(
    retriever,
    qna_chain
)

In [15]:
retriever.invoke("what is acne?")

[Document(metadata={'source': 'kirti pogra'}, page_content='This project is built by Kirti Pogra using LangChain, Pinecone and Groq. The code is available on GitHub. This project is only for educational purposes.')]

In [14]:
async for chunk in rag_chain.astream({"input": "What is acne?"}):
    text = chunk.get('answer')
    if text:
        print(text.content, end="",flush=True)
    

[2025-09-20 16:07:04,867|(INFO)| File: _client | Message: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"]
I'm not sure if that's directly related to the context provided. The context is about a project built by Kirti Pogra using specific technologies, and the code is available on GitHub. However, I can try to provide a general answer about acne as it might be somewhat related to the human body, which might be implied in the context of someone who could be working on a health-related project.

Acne is a common skin condition characterized by the occurrence of pimples, blackheads, and whiteheads. It is caused by a combination of factors, including:

- Overproduction of sebum, an oily substance produced by the sebaceous glands in the skin
- Blockage of the pores by dead skin cells and other debris
- Bacteria buildup, specifically Propionibacterium acnes (P. acnes), which can cause inflammation and lead to acne lesions

According to the American Academ

In [19]:

async def async_stream(query):
    async for chunk in rag_chain.astream({"input": query}):
        text = chunk.get('answer')
        if text:
            word = text.content
            yield word


In [20]:

print("Streaming via astream_events:")
async for token in async_stream("who built this project?"):
    print(token, end="", flush=True)

Streaming via astream_events:
[2025-09-19 21:03:19,605|(INFO)| File: _client | Message: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"]
Based on the provided context, the project was built by Kirti Pogra. According to the metadata, the project's source is listed as 'kirti pogra'. Additionally, the page content mentions that the project is built using LangChain, Pinecone, and Groq, and that the code is available on GitHub. This information suggests that Kirti Pogra is the developer behind this project, and it's intended for educational purposes.

In [ ]:
chat_handler = ChatStreamingHandler()   